In [ ]:
from dotenv import load_dotenv

_ = load_dotenv()

from api.core.infisical import InfisicalManagedCredentials

secrets_client = InfisicalManagedCredentials()

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
)
from typing import Union

from langchain_cohere.chat_models import ChatCohere
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_groq.chat_models import ChatGroq
from api.services.embeddings_factory import EmbeddingsFactory
from api.services.vector_store_factory import VectorStoreFactory
from api.services.llm_factory import LLMFactory
from api.services.memory_factory import MemoryFactory
import api.config.constant as constant
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory.chat_message_histories.upstash_redis import (
    UpstashRedisChatMessageHistory,
)
from langchain_astradb.chat_message_histories import AstraDBChatMessageHistory

In [ ]:
embeddings = EmbeddingsFactory().get_embeddings(
    "sentence-transformers", "intfloat/multilingual-e5-large-instruct"
)

vector_store = VectorStoreFactory().get_vectorstore(
    vectorstore_service="astradb",
    embeddings=embeddings,
)

In [ ]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    chat_history: str
    answer: str
    session_id: str
    category: str
    sub_category: str
    source: str
    memory_service: str
    model_name: str
    temperature: float
    top_k: int
    model: Union[ChatCohere, ChatGoogleGenerativeAI, ChatMistralAI, ChatGroq]
    memory_instance: Union[UpstashRedisChatMessageHistory, AstraDBChatMessageHistory]

In [ ]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=["chat_history", "context", "question"],
                input_types={},
                partial_variables={},
                template=constant.SYSTEM_PROMPT,
            ),
            additional_kwargs={},
        )
    ],
)

In [ ]:
def retrieve(state: State):
    filters = {
        "category": state.get("category"),
        "sub_category": state.get("sub_category"),
    }
    clean_filter = {k: v for k, v in filters.items() if v}
    retrieved_docs = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": state.get("top_k"),
            "filter": clean_filter,
        },
    ).get_relevant_documents(
        query=state["question"],
    )
    print("RETRIEVED DOCS:", retrieved_docs)
    
    def _parse_and_flatten_memory(messages: list):
        memory_string = ""
        for message in messages:
            if type(message) == HumanMessage:
                memory_string += f"Human: {message.content}\n"
            elif type(message) == AIMessage:
                memory_string += f"Assistant: {message.content}\n\n"
        return memory_string

    memory_instance = MemoryFactory().get_memory_instance(
        memory_service=state.get("memory_service"),
        session_id=state.get("session_id"),
    )

    chat_history = _parse_and_flatten_memory(
        memory_instance.messages
    )
    model = LLMFactory.get_chat_model(
        model_name=state.get("model_name"),
        temperature=state.get("temperature", 0.0),
    )
    return {"context": retrieved_docs, "chat_history": chat_history, "model": model, "memory_instance": memory_instance}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    model = state.get("model")
    messages = prompt.invoke(
        {
            "question": state["question"],
            "context": docs_content,
            "chat_history": state["chat_history"],
        }
    )
    response = model.invoke(messages)
    return {"answer": response.content}

def add_message_history(state: State):
    memory_instance = state.get("memory_instance")
    memory_instance.add_user_message(state.get("question"))
    memory_instance.add_ai_message(state.get("answer"))
    return state

In [ ]:
from langgraph.graph import START, StateGraph, END

graph_builder = StateGraph(State).add_sequence([retrieve, generate, add_message_history])
graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("retrieve", "generate")
graph_builder.add_edge("generate", "add_message_history")
graph_builder.add_edge("add_message_history", END)
graph = graph_builder.compile()

In [ ]:
result = graph.invoke(
    {
        "question": "How to add 2D sprites in godot 4?",
        "session_id": "8237648732647238",
        "category": "tutorials",
        "sub_category": None,
        "memory_service": "astradb",
        "model_name": "command-r-plus-08-2024",
        "temperature": 0.5,
        "top_k": 5,
    }
)

print(f"Context: {result['context']}")
print(f"Answer: {result['answer']}")

In [3]:
from api.utils.rtd_reader import ReadTheDocsReader

rtd_loader = ReadTheDocsReader()
docs = rtd_loader.load(directory="./dataset/rtdocs/docs.godotengine.org/en/latest")

2025-10-24 14:14:57.368 | INFO     | api.utils.rtd_reader:__init__:14 - ReadTheDocsReader initialized.


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 1231176: character maps to <undefined>